In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Bronze Process") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.4.1") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9870") \
    .getOrCreate()

## Truy xuất dữ liệu từ mongodb để tìm ra danh sách các id

In [11]:
username = 'ndtien'
password = 'ndtien'
host     = 'mongodb'
port     = '27017'
db       = 'movies_db'
collection = 'movies_name'

MONGO_URL = f"mongodb://{username}:{password}@{host}:{port}/{db}.{collection}?authSource=admin"

In [12]:
movie_id_df = spark.read\
            .format("mongodb")\
            .option("spark.mongodb.read.connection.uri",MONGO_URL)\
            .option('header', 'true')\
            .load()

In [13]:
list_id = movie_id_df.select('id').rdd.flatMap(lambda x: x).collect()

In [14]:
import requests
API_KEY = "2d6e1b290dabf74f65b84431677db2b8"
GET_URL = "https://api.themoviedb.org/3/movie"


def get_movie_data(API_KEY, GET_URL, list_ids):
    list_movies = []
    for movie_id in list_ids:
        url = f'{GET_URL}/{movie_id}?api_key={API_KEY}'
        response = requests.get(url)
        data = response.json()
        list_movies.append(data)
    return list_movies

In [15]:
movie_data = get_movie_data(API_KEY=API_KEY, GET_URL=GET_URL, list_ids=list_id)

In [16]:
movie_data = spark.createDataFrame(movie_data)

In [17]:
movie_data.write.mode("overwrite").format("parquet").save("hdfs://namenode:8020/movies_data/bronze.parquet")

In [18]:
spark.stop()